## Flux Analysis (Relative to Citrate Synthase)

In [5]:
import pandas as pd

#Get steadystate flux output file into a pandas dataframe
ssflux_df =pd.read_csv("./Output_SteadyState_AllValues.csv", index_col=['Biol.Rep','GroupLevel1'])

def weight_calc(x, df):
    '''
    '''
    return df[x].groupby(level=[1]).transform('std')


ssflux_df['std_fACLy'] = weight_calc('fACLy', ssflux_df)
ssflux_df['std_fIDHr'] = weight_calc('fIDHr', ssflux_df)
ssflux_df['std_fPC&MEr'] = weight_calc('fPC&MEr', ssflux_df)
ssflux_df['std_fTCA'] = weight_calc('fTCA', ssflux_df)
ssflux_df['std_fIDH'] = weight_calc('fIDH', ssflux_df)
ssflux_df['std_fAAO'] = weight_calc('fAAO', ssflux_df)
ssflux_df['std_fAADil'] = weight_calc('fAADil', ssflux_df)
ssflux_df['std_fMDHr'] = weight_calc('fMDHr', ssflux_df)
ssflux_df['std_AAS/AAO'] = weight_calc('AAS/AAO', ssflux_df)


    

### Minimization function

In [6]:
import numpy as np
from scipy.optimize import minimize

class Minimizer:
    '''
    '''
    def __init__(self, df, var_names, s_var_names):
        '''
        Initializes the Minimizer object.

        :param df: pandas DataFrame with data to be minimized.
        :param var_names: list of variable names to be optimized.
        :param s_var_names: list of secondary variable names to be optimized.
        '''
        self.df = df
        self.var_names = var_names
        self.s_var_names = s_var_names
    
    def minimizer(self, obj, x0, bounds,cons):
        '''
        Minimizes the objective with initial values and the bounds.

        Parameters:
        obj (callable): The objective function to be minimized.
        x0 (array-like): The initial values for the minimum.
        bounds (sequence): The bounds on the variables.

        Returns:
        OptimizeResult: The result of minimized objective solution for the given initial values
        '''
            
        return minimize(obj, x0=x0, method='SLSQP', bounds=bounds, constraints=cons, jac='2-point', options= {'maxiter':1000})
    
    def recursive_minimizer(self, obj, x0, bounds,cons):
        '''
        Recursively applies a minimization function to refine the solution until a minimum is reached.
    
        Parameters:
        obj (callable): The objective function to be minimized.
        x0 (array-like): The initial estimate for the minimum.
        bounds (sequence): The bounds on the variables.

        Returns:
        OptimizeResult: The result of the final iteration of the minimization function.
        
        '''
        sol = self.minimizer(obj, x0, bounds,cons)
        if sol.success:
            sol2= self.minimizer(obj, sol.x, bounds,cons)
            if round(sol2.fun,2) < round(sol.fun,2):
                return self.recursive_minimizer(obj, sol2.x, bounds,cons)
            else:
                return sol
        else:
            return sol
    
    def run_minimizer(self, n_iter=50):
        '''
        Minimizes the data for each row of the input dataframe.

        :param n_iter: number of iterations for the minimization (default: 50).
        :return: pandas DataFrame with final values and objective function values for each initial value.
        '''
        results = dict()
        for i, row in self.df.iterrows():
            counter = 0
            res_dict = dict()
            
            # defining objective function with row-specific values
            obj_func_row = lambda x: objective_function(x, row)
            
            # defining constraints: All flux_sec_variables(x,row) must be greater than or equal to zero
            con1=[{'type':'ineq','fun': lambda x: flux_sec_variables(x,row)[i] - 0} for i in range(len(self.s_var_names))]
            #con2=[{'type':'ineq','fun': lambda x: enrichment_predict(x,row)[i] - 0} for i in range(9)]
            cons=con1#.append(con2)
            
            # perform minimization with random initial values
            for j in nextVal():
                x0 = np.random.uniform(low=0.01, high=10, size=len(self.var_names))
                bounds = [(0, 10)]*len(self.var_names)
                res = self.recursive_minimizer(obj_func_row, x0, bounds,cons)
                if res.success: 
                    result_row = {self.var_names[k]: res.x[k] for k in range(len(self.var_names))}
                    sec_fluxes = flux_sec_variables(res.x,row)
                    result_row.update({self.s_var_names[k]: sec_fluxes[k] for k in range(len(self.s_var_names))})
                    result_row['objective'] = res.fun
                    res_dict[counter+1] = result_row
                    counter += 1
                    if counter == n_iter:
                        print(i, 'Solution found!')
                        break
                else:
                    counter += 1
                    if counter == n_iter:
                        print(i, 'Solution NOT found!',res.message)
                        break
                    continue
            results[i] = res_dict
        if any(results.values()):
            results_df=pd.DataFrame.from_dict({(*a, b): results[a][b] for a in results.keys() for b in results[a].keys()},orient="index")
            results_df.index.names=["Biol.Rep","GroupLevel1","Iteration"] #add "GroupLevel2", if necessary
            return results_df
        else:
            return pd.DataFrame()


def nextVal(start=1):
    '''
    Generator function that returns an infinite sequence of numbers starting from the specified value.

    Args:
    - start: integer, optional (default=1)
        The starting value of the sequence.

    Yields:
    - val: integer
        The next value in the infinite sequence.
    '''
    val = start
    while True:
        yield val
        val += 1

def combined_dataframe_generator(df,p_enr_names,p_flux_names,balance):
    '''
    Generates a combined dataframe by predicting enrichment and flux ratio using the given input dataframe.
    
    Parameters:
    -----------
    df: Input dataframe
    p_enr_names: List of names for the predicted enrichment values
    p_flux_names: List of names for the predicted flux ratio values
    
    Returns:
    --------
    pandas.DataFrame
        Combined dataframe with predicted enrichment and flux ratio values added as additional columns
    '''
    x = [df[i] for i in var_names]
    predict_enr = enrichment_predict(x, df)
    predict_fluxratio = fluxratio_predict(x, df)
    bal = ss_balance(x,df)
    
    p_enrichment_df = pd.DataFrame(predict_enr, index=p_enr_names).T
    p_fluxratio_df  = pd.DataFrame(predict_fluxratio, index=p_flux_names).T
    p_ssbalance_df  = pd.DataFrame(bal, index=balance).T
    
    return pd.concat([df,p_enrichment_df,p_fluxratio_df,p_ssbalance_df],axis=1)

### Assigning functions for
1. Secondary fluxes (Secondary variables)
2. Predicted enrichments
3. Predicted steady state flux ratios
4. Objective solution to minimize

In [7]:
def flux_sec_variables(x,A):
    '''
    Dependent floating fluxes
    '''
    [anap,acly,tca,mef,mdhr,idh,idhr,aao,aas,aadil,gln_entry]= x
    
    #Secondary flux variables
    cs = 1
    pdh = A['fPDH_average']
    pck =0
    
    return cs,pdh,pck

######

def enrichment_predict(x, A):
    '''
    Predicted enrichments
    '''
    [anap,acly,tca,mef,mdhr,idh,idhr,aao,aas,aadil,gln_entry]= x
    [cs,pdh,pck] = flux_sec_variables(x,A)

    #4-13C Glc predicted enrichments
    f_mdhr = mdhr/(mdhr+anap+tca)

    pMal1  = (A['4g3PGM1']*anap*(1-f_mdhr) + A['4gBicarb']*anap*f_mdhr + A['Pred_4gCit6']*A['FCitDil_average']*acly*(1-f_mdhr) + A['Pred_4gCit1']*A['FCitDil_average']*acly*f_mdhr)/(acly+anap+tca)
    pMal4  = (A['4g3PGM1']*anap*f_mdhr + A['4gBicarb']*anap*(1-f_mdhr) + A['Pred_4gCit6']*A['FCitDil_average']*acly*f_mdhr + A['Pred_4gCit1']*A['FCitDil_average']*acly*(1-f_mdhr))/(acly+anap+tca)
    pMalM1 = pMal1 + pMal4
    pGluM1 = (A['Pred_4gCit1']*A['FCitDil_average']*idh + A['4gGlnM1']*aao)/(idh+aao+aadil)
    pGlnM1 = (A['4gGluM1']*aas)/(aas+gln_entry)

    
    return f_mdhr,pMal1,pMal4,pMalM1,pGluM1,pGlnM1

######
    
def fluxratio_predict(x, A):
    '''
    Predicted flux ratios
    '''
    [anap,acly,tca,mef,mdhr,idh,idhr,aao,aas,aadil,gln_entry]= x
    [cs,pdh,pck] = flux_sec_variables(x,A)

    #Predicted flux ratios
    pf_acly     = acly/(anap+tca+acly)
    pf_idhr     = idhr/(idhr+cs)
    pf_anap     = anap/(anap+tca+acly)
    pf_tca     = tca/(anap+tca+acly)
    pf_idh      = idh/(idh+aao+aadil)
    pf_aao      = aao/(idh+aao+aadil)
    pf_aadil    = aadil/(idh+aao+aadil)
    pf_mdhr    = mdhr/(mdhr+anap+tca)
    pf_aas    = aas/aao
    
    return pf_acly,pf_idhr,pf_anap,pf_tca,pf_idh,pf_aao,pf_aadil,pf_mdhr,pf_aas

######
  
def ss_balance(x,A):
    '''
    '''
    [anap,acly,tca,mef,mdhr,idh,idhr,aao,aas,aadil,gln_entry]= x
    [cs,pdh,pck] = flux_sec_variables(x,A)
    f_mdhr,pMal1,pMal4,pMalM1,pGluM1,pGlnM1 = enrichment_predict(x,A)
    pf_acly,pf_idhr,pf_anap,pf_tca,pf_idh,pf_aao,pf_aadil,pf_mdhr,pf_aas = fluxratio_predict(x,A)

    #Anaplerosis - Cataplerosis flux difference
    ss_d1 = abs((cs+idhr)-(idh+acly))
    ss_d2 = abs((anap+acly+tca)-(cs+pck+mef))
    ss_d3 = abs((idh+aao+aadil)-(tca+idhr+aas+aadil))
    ss_d4 = abs((anap+aao)-(mef+pck+aas))
   
    return ss_d1,ss_d2,ss_d3,ss_d4

######

def objective_function(x, A):
    '''
    Objective function to minimize
    '''
    [anap,acly,tca,mef,mdhr,idh,idhr,aao,aas,aadil,gln_entry]= x
    [cs,pdh,pck] = flux_sec_variables(x,A)
    f_mdhr,pMal1,pMal4,pMalM1,pGluM1,pGlnM1 = enrichment_predict(x,A)
    pf_acly,pf_idhr,pf_anap,pf_tca,pf_idh,pf_aao,pf_aadil,pf_mdhr,pf_aas = fluxratio_predict(x,A)
    ss_d1,ss_d2,ss_d3,ss_d4 = ss_balance(x,A)

    #Anaplerosis - Cataplerosis flux difference
    diff1 = sum([ss_d1,ss_d2,ss_d3,ss_d4])

    
    #Predicted - Measured enrichment difference
    dMalM1   = abs(pMalM1 - A["4gMalM1"])
    dGluM1   = abs(pGluM1 - A["4gGluM1"])
    dGlnM1   = abs(pGlnM1 - A["4gGlnM1"])

    diff2    = sum([dMalM1,dGluM1,dGlnM1])*10

    #Predicted - Measured fluxratio difference
    df_acly   = abs(pf_acly - A['fACLy_average']*(1+A['std_fACLy']))
    df_acly2   = abs(pf_acly - A['fACLy_average']*(1-A['std_fACLy']))
    df_idhr   = abs(pf_idhr - A['fIDHr_average']*(1+A['std_fIDHr']))
    df_idhr2   = abs(pf_idhr - A['fIDHr_average']*(1-A['std_fIDHr']))
    df_anap   = abs(pf_anap - A['fPC&MEr_average']*(1+A['std_fPC&MEr']))
    df_anap2   = abs(pf_anap - A['fPC&MEr_average']*(1-A['std_fPC&MEr']))
    df_tca    = abs(pf_tca - A['fTCA_average']*(1+A['std_fTCA']))
    df_tca2    = abs(pf_tca - A['fTCA_average']*(1-A['std_fTCA']))
    df_idh    = abs(pf_idh - A['fIDH_average']*(1+A['std_fIDH']))
    df_idh2    = abs(pf_idh - A['fIDH_average']*(1-A['std_fIDH']))
    df_aao    = abs(pf_aao - A['fAAO_average']*(1+A['std_fAAO']))
    df_aao2    = abs(pf_aao - A['fAAO_average']*(1-A['std_fAAO']))
    df_aadil  = abs(pf_aadil - A['fAADil_average']*(1+A['std_fAADil']))
    df_aadil2  = abs(pf_aadil - A['fAADil_average']*(1-A['std_fAADil']))
    df_mdhr   = abs(pf_mdhr - A['fMDHr_average']*(1+A['std_fMDHr']))
    df_mdhr2   = abs(pf_mdhr - A['fMDHr_average']*(1-A['std_fMDHr']))
    df_aas    = abs(pf_aas - A['AAS/AAO_average']*(1+A['std_AAS/AAO']))
    df_aas2    = abs(pf_aas - A['AAS/AAO_average']*(1-A['std_AAS/AAO']))
    
    diff3     = sum([df_acly,df_acly2,df_idhr,df_idhr2,df_anap,df_anap2,df_tca,df_tca2,df_idh,df_idh2,df_aao,df_aao2,df_aadil,df_aadil2,df_mdhr,df_mdhr2,df_aas,df_aas2])
    
    return sum([diff1,diff2,diff3])

### Declaring the names of variables and predictions

In [8]:
#List of floating variables
var_names    = ['anap','acly','tca','mef','mdhr','idh','idhr','aao','aas','aadil','gln_entry']

#List of secondary(dependent) variables
s_var_names  = ['cs','pdh','pck']

#List of predicted enrichments
p_enr_names  = ['f_mdhr','pMal1','pMal4','pMalM1','pGluM1','pGlnM1']

#List of predicted steady state flux ratios
p_flux_names = ['pf_acly','pf_idhr','pf_anap','pf_tca','pf_idh','pf_aao','pf_aadil','pf_mdhr','pf_aas']

#List of flux balance
balance = ['ss_d1','ss_d2','ss_d3','ss_d4']

### Minimization 

In [11]:
import warnings
warnings.filterwarnings("ignore")

print('Starting Minimization...')
#Initializing the Minimizer with input dataframe(ssflux output) and floating and dependent variables
minimizer = Minimizer(ssflux_df, var_names, s_var_names)

#Running the minimizer with desired no. of iteration (Default = 50)
var_results_df = minimizer.run_minimizer(n_iter= 50)

#Storing the results into a dataframe with adding iteration column to the index
#add , "GroupLevel2" if necessary
results_df = pd.merge(ssflux_df, var_results_df.reset_index(level=[2]), on=["Biol.Rep", "GroupLevel1"], how="outer").set_index(["Iteration"], append=True)

#Combining the minimized results with final predicted enrichments and flux ratios
combined_results = combined_dataframe_generator(results_df,p_enr_names,p_flux_names,balance)

print('Minimization Completed')
combined_results

Starting Minimization...
(1.0, 'G2.5Q2') Solution found!
(2.0, 'G2.5Q2') Solution found!
(3.0, 'G2.5Q2') Solution found!
(4.0, 'G2.5Q2') Solution found!
(1.0, 'G5Q2') Solution found!
(2.0, 'G5Q2') Solution found!
(3.0, 'G5Q2') Solution found!
(4.0, 'G5Q2') Solution found!
(1.0, 'G7Q2') Solution found!
(2.0, 'G7Q2') Solution found!
(3.0, 'G7Q2') Solution found!
(4.0, 'G7Q2') Solution found!
(1.0, 'G9Q2') Solution found!
(2.0, 'G9Q2') Solution found!
(3.0, 'G9Q2') Solution found!
(4.0, 'G9Q2') Solution found!
Minimization Completed


qqAspM1   qqAspM2   qqCitM1   qqCitM2  \
Biol.Rep GroupLevel1 Iteration                                           
1.0      G2.5Q2      1          0.441466  0.030876  0.279632  0.126028   
                     2          0.441466  0.030876  0.279632  0.126028   
                     3          0.441466  0.030876  0.279632  0.126028   
                     4          0.441466  0.030876  0.279632  0.126028   
                     5          0.441466  0.030876  0.279632  0.126028   
...                                  ...       ...       ...       ...   
4.0      G9Q2        45         0.260998  0.024073  0.221323  0.072546   
                     47         0.260998  0.024073  0.221323  0.072546   
                     48         0.260998  0.024073  0.221323  0.072546   
                     49         0.260998  0.024073  0.221323  0.072546   
                     50         0.260998  0.024073  0.221323  0.072546   

                                 qqCitM3   qqGluM1   qqGluM2   qqGluM3  \
Biol.Rep GroupLevel1 Iteration                                           
1.0      G2.5Q2      1          0.008214  0.059144  0.545513  0.022117   
                     2          0.008214  0.059144  0.545513  0.022117   
                     3          0.008214  0.059144  0.545513  0.022117   
                     4          0.008214  0.059144  0.545513  0.022117   
                     5          0.008214  0.059144  0.545513  0.022117   
...                                  ...       ...       ...       ...   
4.0      G9Q2        45         0.005305  0.053112  0.427825  0.017706   
                     47         0.005305  0.053112  0.427825  0.017706   
                     48         0.005305  0.053112  0.427825  0.017706   
                     49         0.005305  0.053112  0.427825  0.017706   
                     50         0.005305  0.053112  0.427825  0.017706   

                                 qqGlnM2   qqGlnM3  ...    pf_tca    pf_idh  \
Biol.Rep GroupLevel1 Iteration                      ...                       
1.0      G2.5Q2      1          0.836706  0.040748  ...  0.874151  0.280637   
                     2          0.836706  0.040748  ...  0.864074  0.284705   
                     3          0.836706  0.040748  ...  0.875984  0.284257   
                     4          0.836706  0.040748  ...  0.875684  0.281906   
                     5          0.836706  0.040748  ...  0.871531  0.277950   
...                                  ...       ...  ...       ...       ...   
4.0      G9Q2        45         0.810522  0.038665  ...  0.646801  0.378206   
                     47         0.810522  0.038665  ...  0.646804  0.378203   
                     48         0.810522  0.038665  ...  0.646742  0.378287   
                     49         0.810522  0.038665  ...  0.646817  0.378422   
                     50         0.810522  0.038665  ...  0.646391  0.378225   

                                  pf_aao  pf_aadil   pf_mdhr    pf_aas  \
Biol.Rep GroupLevel1 Iteration                                           
1.0      G2.5Q2      1          0.590442  0.128922  0.348440  0.237539   
                     2          0.586936  0.128358  0.352541  0.243637   
                     3          0.586403  0.129341  0.383944  0.234308   
                     4          0.590233  0.127861  0.371690  0.242702   
                     5          0.594499  0.127551  0.385450  0.250969   
...                                  ...       ...       ...       ...   
4.0      G9Q2        45         0.480020  0.141774  0.405281  0.276679   
                     47         0.479205  0.142593  0.406239  0.272782   
                     48         0.478756  0.142957  0.402901  0.272684   
                     49         0.481095  0.140482  0.404840  0.277058   
                     50         0.478742  0.143033  0.403596  0.274907   

                                       ss_d1         ss_d2         ss_d3  \
Biol.Rep GroupLevel1 Iteration          

In [10]:
combined_results.to_csv('Output_SteadyState_FluxesRelativeCS_AllIterations.csv')
ResultsFinal = combined_results.groupby(level=[0,1]).median()
ResultsFinal.to_csv('Output_SteadyState_FluxesRelativeCS.csv')
